<table width="100%">
    <tr style="border-bottom:solid 2pt #009EE3">
        <td class="header_buttons">
            <a href="gon_angular_velocity.zip" download><img src="../../images/icons/download.png" alt="biosignalsnotebooks | download button"></a>
        </td>
        <td class="header_buttons">
            <a href="https://mybinder.org/v2/gh/biosignalsplux/biosignalsnotebooks/mybinder_complete?filepath=biosignalsnotebooks_environment%2Fcategories%2FExtract%2Fgon_angular_velocity.dwipynb" target="_blank"><img src="../../images/icons/program.png" alt="biosignalsnotebooks | binder server" title="Be creative and test your solutions !"></a>
        </td>
        <td></td>
        <td class="header_icons">
            <a href="../MainFiles/biosignalsnotebooks.ipynb"><img src="../../images/icons/home.png" alt="biosignalsnotebooks | home button"></a>
        </td>
        <td class="header_icons">
            <a href="../MainFiles/contacts.ipynb"><img src="../../images/icons/contacts.png" alt="biosignalsnotebooks | contacts button"></a>
        </td>
        <td class="header_icons">
            <a href="https://github.com/biosignalsplux/biosignalsnotebooks" target="_blank"><img src="../../images/icons/github.png" alt="biosignalsnotebooks | github button"></a>
        </td>
        <td class="header_logo">
            <img src="../../images/ost_logo.png" alt="biosignalsnotebooks | project logo">
        </td>
    </tr>
</table>

<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_6"><div id="image_img"
        class="header_image_6"></div></td>
        <td class="header_text"> GON - Angular velocity estimation </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">extract&#9729;gon&#9729;angle&#9729;velocity&#9729;acceleration</td>
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

Goniometers are instruments that measure the relative angle between two points. In the case of <a href='https://plux.info/sensors/290-goniometer-gon-820202502.html' target="_blank">PLUX's Goniometer (GON) sensor <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>, the measurement is made between two plaques. 

Given the temporal <b class='color4'>evolution of the angle</b> between the two plaques, through the application of simple physical principles, it is possible to calculate both <strong><span class="color13">angular velocity</span></strong> and <strong><span class="color7">angular acceleration</span></strong>.

In this <b class='color11'>Jupyter Notebook</b>, we will demonstrate how to calculate the <b class='color2'>angular velocity</b> starting with a signal acquired using the <b>Goniometer</b> and how to compute those values using <b>Python</b>.

<hr>

<p class="steps">1 - Import the required packages</p>
<strong class="color1">First, we will import the required Python packages that will facilitate our job later in this notebook.</strong>

In [1]:
# Import the biosignalsnotebooks Python package
import biosignalsnotebooks as bsnb

# Import some functions of the numpy Python package that will help us in the mathematical steps of the notebook
from numpy import diff

<p class="steps">2 - Load the GON signal</p>
For example purposes, we will use a previously acquired signal.

In [2]:
# Define the path of the file that has the signals of each device
file_name = '../../signal_samples/GON.h5'

# Load the data from the file
acquisition, header = bsnb.load(file_name, get_header=True)

# Get the signal that was acquired in the channel 2 of the device
gon_signal = acquisition['CH2']

# Get the sampling rate of the acquisition
sr = header['sampling rate']

In [3]:
# Generate the time axis
time_axis = bsnb.generate_time(gon_signal, sample_rate=sr)

# lowPass filter to get rid of noise
gon_signal = bsnb.lowpass(gon_signal, f=1, fs=sr, use_filtfilt=True)

# Plot the signal
bsnb.plot(time_axis, gon_signal, y_axis_label="Raw Units (a.u.)", title='Goniometer Signal')

<p class="steps">2.1 - Conversion to Physical Units</p>
In order to convert the signal to its physical units, we need to apply the transfer function for the <strong>Goniometer</strong> sensor that is present in the <a href='https://biosignalsplux.com/datasheets/GON_Sensor_Datasheet.pdf#page=2' target="_blank">sensor datasheet<img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>.

<span class='color7'><strong>Note:</strong></span> If you find this step troubling, we invite you to read the <a href='https://biosignalsplux.com/notebooks/Categories/Pre-Process/unit_conversion_gon_rev.php' target='_blank' class='color2'>Goniometer Sensor - Unit Conversion <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>

In [4]:
# Vcc value of the device
vcc = 3

# Resolution used during the acquisition (the index 1, serves to get the resolution of CH2)
resolution = header['resolution'][1]

# Applying the transfer function to the signals
gon_signal_converted = ((vcc / (2**resolution - 1)) * gon_signal - (vcc / 2)) / ((vcc / 2) * 606e-5)

In [5]:
# Plot the signal
bsnb.plot(time_axis, gon_signal_converted, y_axis_label="Degrees (\u00BA)", title='Goniometer Signal')

<p class="steps">3 - Calculate the angular velocity</p>
<strong class="color1">To calculate the angular velocity given the variation of the angle over time, we need to apply a math operation called derivative.</strong>

In physics, a velocity of some sort corresponds to the variation of a given variable <i>over time</i> ($dt$). In this case, <strong class='color4'>angular velocity</strong> ($\omega$), as the name indicates, corresponds to the variation of an angle ($d\theta$). 

A variation can be represented as the derivative of a function or a signal, if the time interval is short enough, and is usually represented as:

$$ \omega = \frac{d\theta}{dt} $$

In practical terms, the time between samples is sufficient to apply the simplified/discrete version, in which the velocity corresponds to the mean velocity between two points in time. Thus, in this case it is represented as: 

$$ \omega = \frac{\Delta\theta}{\Delta t} = \frac{\theta_i - \theta_{i-1}}{t_i - t_{i-1} } $$

Thus, the velocity in each point corresponds to the difference between the present value ($\theta_i$) and the previous one ($\theta_{i-1}$) divided by the present time instant ($t_i$) minus the previous one ($t_{i-1}$). The variation of time corresponds then to the period of acquisition, which is the time between two consecutive samples.

Hence, to calculate the velocity, we first need to calculate the period of acquisition, that is the inverse of the sampling rate.

In [6]:
period_acquisition = 1/sr

Then, we need to calculate the difference between consecutive samples to divide by the period of acquisition. Fortunately, numpy has that same function implemented and, so, we only need to use it.

In [7]:
diff_consecutive_samples = diff(gon_signal_converted)

The last step left includes the calculation of the angular velocity. This step consists on dividing the difference between consecutive samples (```diff_consecutive_samples```) by the time between consecutive samples (```period```).

In [8]:
angular_velocity = diff_consecutive_samples / period_acquisition

In [9]:
time_velocity = bsnb.generate_time(angular_velocity)
bsnb.plot(time_velocity, angular_velocity, y_axis_label="Angular Velocity (\u00BA/s)", title='Angular Velocity Signal')

The same could be made to calculate the <strong><span class="color7">angular acceleration</span></strong>: calculate the difference between consecutive points of the <strong><span class="color13">angular velocity</span></strong> signal and divide by the period of the acquisition to end up with the angular acceleration signal.

In this notebook, you learned how to calculate the angular velocity given the angle over time, by calculating the (<it>pseudo</it>) derivative of the signal. Thus, you can apply the same rationale to calculate velocity for every signal that represents position, angle or other over time.

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> ! 

<hr>
<table width="100%">
    <tr>
        <td class="footer_logo">
            <img src="../../images/ost_logo.png" alt="biosignalsnotebooks | project logo [footer]">
        </td>
        <td width="40%" style="text-align:left">
            <a href="../MainFiles/aux_files/biosignalsnotebooks_presentation.pdf" target="_blank">&#9740; Project Presentation</a>
            <br>
            <a href="https://github.com/biosignalsplux/biosignalsnotebooks" target="_blank">&#9740; GitHub Repository</a>
            <br>
            <a href="https://pypi.org/project/biosignalsnotebooks/" target="_blank">&#9740; How to install biosignalsnotebooks Python package ?</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/signal_samples.ipynb">&#9740; Signal Library</a>
        </td>
        <td width="40%" style="text-align:left">
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/biosignalsnotebooks.ipynb">&#9740; Notebook Categories</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/by_diff.ipynb">&#9740; Notebooks by Difficulty</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/by_signal_type.ipynb">&#9740; Notebooks by Signal Type</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/by_tag.ipynb">&#9740; Notebooks by Tag</a>
        </td>
    </tr>
</table>

In [10]:
from biosignalsnotebooks.__notebook_support__ import css_style_apply
css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................


In [11]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'],
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>